## Dubai Local Travel AI Chatbot Assist

<img src="dubai_local_tours.jpeg" style="display:block; margin-left:auto; margin-right:auto;"/>

### Problem Statement

Dubai, as one of the world’s most visited cities, presents a rich blend of attractions—cultural landmarks, modern entertainment hubs, shopping malls, beaches, and more. However, **travelers often struggle** to explore the city efficiently due to:

- **Overwhelming choices** with minimal personalization.

- **Language barriers**, especially with Arabic or mixed cultural signs and local dialects.

- **Lack of a single intelligent interface** that can help plan itineraries, provide local info, answer questions, and adjust dynamically.

- Difficulty in navigating real-time changes (e.g., weather, events, closures).

- The absence of context-aware digital guides who understand individual travel preferences.

Traditional apps (like Google Maps or TripAdvisor) are fragmented and reactive. Tourists still rely heavily on manual research, guidebooks, or static platforms that don’t adapt or personalize effectively.

### Project Overview

The Dubai Local Explorer AI Chatbot is a GPT-powered, context-aware virtual assistant designed to guide tourists in Dubai through real-time conversational interactions. It brings together the intelligence of natural language processing (NLP), real-time APIs, and curated city data to help users:

- Discover places to visit.

- Get personalized itineraries.

- Receive live assistance (navigation, translation, recommendations).

- Learn about local culture, rules, and tips.

The system runs through a chat interface and is designed to be modular and extensible—ideal for deployment across mobile apps, web apps, or messaging platforms (WhatsApp, Telegram, etc.).

### Project Objectives

**Traveler-centric Goals**

- Deliver contextual recommendations based on interest (e.g., art, adventure, food).

- Serve as an always-available digital concierge.

- Improve cultural understanding and navigation through translation and etiquette help.

**System-level Objectives**

- Utilize LLM-based dialogue management to maintain engaging, multi-turn conversations.

- Integrate with external data providers/APIs for up-to-date weather, events, transport, and venue information.

- Store and recall user preferences for personalized follow-ups.

- Provide multilingual communication (Arabic, English, Hindi/Urdu).

- Ensure scalability and adaptability for other cities or languages in the future.

### End to End high level workflow

### Functional Features

- Warm welcome message on chat open
- Real-time two-way chatbot interface
- Moderation checks on user input
- Personalized recommendations using function calls
- Rich formatting of results with emojis and details
- Error handling & validation throughout the flow


In [12]:
# Import the libraries
import os, json, ast
import pandas as pd
import openai
from openai import OpenAI
from typing import List, Dict, Union, Any, Tuple

In [13]:
#Let's check the version of OpenAI
import openai
print(openai.__version__)

1.95.1


In [14]:
# Read key from text file
with open("openai_key.txt", "r") as f:
    api_key = f.read().strip()

In [15]:
# Pass key to OpenAI client
client = OpenAI(api_key=api_key)

In [16]:
# Read the dataset and read the Laptop Dataset
dubai_tourist_spots = pd.read_csv('dubai_tours.csv')
dubai_tourist_spots.head()

,name,tags,min_budget,max_budget,duration_hours,suitable_for,description
0,Desert Safari,"adventure,outdoor,desert",300,700,6.0,"family,couple,friends","Dune‑bashing, camel rides, traditional dinner ..."
1,JBR Beach Day,"beach,outdoor,relaxation",0,100,5.0,"family,couple,friends",Relax on sands and enjoy water sports at JBR B...
2,Motiongate Dubai,"theme‑park,Hollywood,rides",150,350,6.0,"family,friends",Hollywood‑inspired rides and shows at Dubai Pa...
3,Dubai Museum Tour,"culture,museum,historical",50,150,2.0,"family,couple,solo",Explore Al Fahidi Fort history exhibitions.
4,Dubai Aquarium & Underwater Zoo,"marine,indoor,family",100,250,3.0,"family,couple,solo","See sharks, rays, and marine life within Dubai..."


In [17]:
dubai_tourist_spots.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   name            100 non-null    object 
 1   tags            100 non-null    object 
 2   min_budget      100 non-null    int64  
 3   max_budget      100 non-null    int64  
 4   duration_hours  100 non-null    float64
 5   suitable_for    100 non-null    object 
 6   description     100 non-null    object 
dtypes: float64(1), int64(2), object(4)
memory usage: 5.6+ KB


### 1. Initialize Conversation for Dubali Local Explorer AI Chatbot

In [19]:
def initialize_conversation():
    """
    Returns a rich initial prompt including:
    - A system message defining assistant personality and constraints
    - Multiple few-shot examples that demonstrate reasoning
    - Sets context for consistent user interaction
    """

    initial_system_prompt = {
        "role": "system",
        "content": (
            "You are DubaiLocalExplorer, an intelligent assistant that helps tourists explore Dubai. "
            "You engage in natural conversation to understand user preferences, confirm intent, and suggest "
            "personalized day-trip itineraries. Use friendly, concise language. "
            "\n\nYour goal is to:\n"
            "- Help users clarify their interests, budget, group type, and duration\n"
            "- Ask clarifying questions if data is missing\n"
            "- Suggest experiences that match those constraints\n"
            "- Think step-by-step and summarize total cost & time\n\n"
            "Always tailor suggestions based on their profile. Keep the tone warm and adaptive."
        )
    }

    dubai_experiences_examples = [
        {
            "role": "user",
            "content": "Hi! I'm in Dubai for 1 day. I'd love to explore cultural sites and try local food."
        },
        {
            "role": "assistant",
            "content": (
                "That sounds like a wonderful day! Here's a quick plan:\n"
                "1. Start at Al Fahidi Historical District to explore Dubai Museum.\n"
                "2. Ride an abra across the Creek to the Gold and Spice Souks.\n"
                "3. Try Emirati cuisine at Al Seef or Arabian Tea House.\n\n"
                "Would you prefer walking between places, or prefer transport arranged?"
            )
        },
        {
            "role": "user",
            "content": "I'm traveling with my wife, and we have around 700 AED to spend."
        },
        {
            "role": "assistant",
            "content": (
                "Thanks for sharing! That gives us good flexibility.\n"
                "I'll make sure all experiences are within budget for a couple. "
                "Would you like to include modern attractions like Dubai Frame or Museum of the Future?"
            )
        },
        {
            "role": "user",
            "content": "We are a group of friends, want nightlife and beaches, budget is tight though."
        },
        {
            "role": "assistant",
            "content": (
                "Got it — you're a group looking for beach and nightlife fun, but mindful of costs.\n"
                "Let me find beach clubs or night spots with free or low entry fees, and suggest group-friendly spots. "
                "Can you tell me how many days you’re staying?"
            )
        },
        {
            "role": "user",
            "content": "Just one full day. Don’t want to waste time in malls."
        },
        {
            "role": "assistant",
            "content": (
                "Understood — one focused day without shopping distractions.\n"
                "How about this:\n"
                "- Morning: Relax at Kite Beach (free entry, lots of cafés)\n"
                "- Afternoon: Explore JBR beachfront and walk The Beach boardwalk\n"
                "- Evening: Rooftop dinner at a budget-friendly lounge near Dubai Marina\n\n"
                "Want me to include transport suggestions?"
            )
        }
    ]

    return [initial_system_prompt] + dubai_experiences_examples


### 2. Get Chat Model Completions using GPT LLM API

In [21]:
def get_chat_model_completions(conversation, model="gpt-3.5-turbo", temperature=0.4, tools=None, tool_choice=None, print_response=True):
    """
    Submits a message list (conversation) to the OpenAI ChatCompletion API and supports tool (function) calls.

    Returns:
        dict: {"type": "message" or "tool_call", "content" or "tool_name"/"tool_args"}
    """
    try:
        # Prepare the base request
        request_payload = {
            "model": model,
            "messages": conversation,
            "temperature": temperature,
        }

        # Only add tools and tool_choice if tools are provided
        if tools:
            request_payload["tools"] = tools
            if tool_choice:
                request_payload["tool_choice"] = tool_choice

        # Make the API call
        response = client.chat.completions.create(**request_payload)

        choice = response.choices[0].message

        # Handle tool (function) call
        if hasattr(choice, "tool_calls") and choice.tool_calls:
            tool_call = choice.tool_calls[0]
            tool_name = tool_call.function.name
            tool_args = json.loads(tool_call.function.arguments)
            tool_call_id = tool_call.id

            if print_response:
                print(f"\n Assistant wants to call: {tool_name}")
                print("With arguments:", tool_args)

            return {
                "type": "tool_call",
                "tool_name": tool_name,
                "tool_args": tool_args,
                "tool_call_id": tool_call_id
            }

        # Handle plain message
        else:
            message = choice.content.strip()
            if print_response:
                print("\n Assistant Response:\n")
                print(message)

            return {
                "type": "message",
                "content": message
            }

    except Exception as e:
        print("OpenAI API error:", e)


In [22]:
conversation = [
    {"role": "system", "content": "You extract user travel preferences into JSON format."},
    {"role": "user", "content": "I'm coming with kids, want beach and outdoor stuff. Budget is around 700 AED."},
    {"role": "user", "content": "Staying for one day only."},
    {"role": "user", "content": (
        "Please extract my travel preferences as structured JSON with keys: "
        "'interests', 'duration_days', 'budget_aed', 'group_type'."
    )}
]

response_from_assistant  = get_chat_model_completions(conversation,tool_choice="auto")


 Assistant Response:

{
  "interests": ["beach", "outdoor activities"],
  "duration_days": 1,
  "budget_aed": 700,
  "group_type": "family with kids"
}


### 3. Moderation Check

In [24]:
def moderation_check(user_input):
    """
    Uses OpenAI's Moderation API to check if input violates content policy.
    Returns a dict with moderation status and flagged categories.
    """
    try:
        response = client.moderations.create(input=user_input)
        result = response.results[0]

        if result.flagged:
            return f"Flagged by moderation. Categories: {result.categories}"
        
        return "Moderation check passed for given input: "+ user_input

    except Exception as e:
        print("Exception occurred:", e)
        return {
            "flagged": False,
            "categories": {},
            "error": str(e)
        }


### 4. Extract information from user message

In [26]:
def extract_information(raw_text):
    """
    Uses LLM to extract structured info (interests, budget, duration, group_type)
    from raw user input.
    """
    messages = [
                {"role": "system", "content": "You are an assistant that extracts structured travel preferences from user input. Return a Python dictionary."},
                {"role": "user", "content": f"""Extract the following from the user's message as a Python dictionary:
                - interests (list of strings)
                - duration_days (integer)
                - budget_aed (integer)
                - group_type (string like 'family', 'couple', 'solo', or 'friends')
                
                User Input: \"\"\"{raw_text}\"\"\""""},
            ]

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            temperature=0.2,
            max_tokens=200
        )
        response_text = response.choices[0].message.content.strip()

        # Safe eval (only if you're confident in the format being returned)
        parsed_dict = ast.literal_eval(response_text)
        if isinstance(parsed_dict, dict):
            return parsed_dict
        else:
            return None

    except Exception as e:
        print(f"Error extracting information: {e}")
        return None


### 5. Product Mapping with internal database

In [28]:
def product_mapping(
    user_profile: Dict[str, Any],
    experiences: Union[pd.DataFrame, List[Dict[str, Any]]],
    min_required_matches: int = 5
) -> List[Dict[str, Any]]:
    """
    Maps user profile to Dubai experiences database, scores each candidate, and returns a
    sorted list (highest score first). Does NOT truncate; pagination / top‑N handled elsewhere.

    Columns expected: ['name', 'tags', 'min_budget', 'max_budget', 'duration_hours', 'suitable_for', 'description']

    user_profile keys: interests (list[str]), budget_aed (int), duration_days (int/float), group_type (str)
    """

    # Convert list-of-dicts to DataFrame if needed
    if isinstance(experiences, list):
        experiences = pd.DataFrame(experiences)
    elif not isinstance(experiences, pd.DataFrame):
        raise ValueError("`experiences` must be a pandas DataFrame or list of dicts")

    # Required columns check
    required = {"name", "tags", "min_budget", "max_budget", "duration_hours", "suitable_for", "description"}
    missing = required - set(experiences.columns)
    if missing:
        raise KeyError(f"Missing {missing} in experiences")

    # Normalize list-like columns
    def ensure_list(x):
        if isinstance(x, str):
            return [s.strip().lower() for s in x.split(",") if s.strip()]
        elif isinstance(x, (list, tuple, set)):
            return [str(s).strip().lower() for s in x]
        else:
            raise TypeError(f"Unexpected type for tags/suitable_for: {x!r}")

    df = experiences.copy()
    df["tags"] = df["tags"].map(ensure_list)
    df["suitable_for"] = df["suitable_for"].map(ensure_list)

    # Profile
    interests = set(i.strip().lower() for i in user_profile.get("interests", []))
    budget = float(user_profile.get("budget_aed", 0))
    duration_days = float(user_profile.get("duration_days", 0))
    group_type = user_profile.get("group_type", "").strip().lower()
    available_hours = duration_days * 8.0 if duration_days > 0 else 0

    # Filter step
    budget_filt = df["min_budget"].le(budget) | df["max_budget"].ge(budget)
    dur_filt = df["duration_hours"].le(max(1, available_hours))
    df = df[ budget_filt & dur_filt ].copy()

    if df.empty:
        return []

    # Scoring function
    def score_row(r):
        score = 0
        parts = []

        tag_overlap = interests.intersection(r["tags"])
        tpts = len(tag_overlap) * 5
        score += tpts
        if tag_overlap:
            parts.append(f"Tags match: {', '.join(sorted(tag_overlap)):5s} (+{tpts})")

        if r["min_budget"] <= budget <= r["max_budget"]:
            score += 20
            parts.append("Budget (+20)")
        else:
            parts.append("Budget (+0)")

        if r["duration_hours"] <= available_hours:
            score += 15
            parts.append("Duration fits (+15)")
        else:
            parts.append("Duration too long (+0)")

        if group_type in r["suitable_for"]:
            score += 10
            parts.append(f"Group '{group_type}' ok (+10)")
        else:
            parts.append(f"Group '{r['suitable_for']}' mismatch (+0)")

        return pd.Series({"score": score, "rationale": "; ".join(parts)})

    scored = df.apply(score_row, axis=1)
    df = df.join(scored)
    df = df.sort_values("score", ascending=False)

    results = (
        df.reset_index(drop=True)
          .loc[:, ["name", "description", "score", "rationale"]]
          .to_dict(orient="records")
    )

    return results


In [29]:
def parse_columns(df):
    for col in ["tags", "suitable_for"]:
        df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    return df

### 6. Match Dubai experiences with given profile

In [31]:
def match_experiences_to_profile(user_profile, experiences=dubai_tourist_spots):
    """
    Match Dubai experiences to user profile with scoring and detailed reasoning.

    Args:
        user_profile (dict): Dictionary with keys:
            - interests (list[str])
            - budget_aed (int)
            - duration_days (int)
            - group_type (str)

        experiences (list[dict]): List of Dubai experiences.

    Returns:
        list of dict: Sorted recommended experiences with scores and reasons.
    """

    # Convert list of dicts into DataFrame if needed
    if not isinstance(experiences, pd.DataFrame):
        experiences = pd.DataFrame(experiences)

    # Required columns
    expected = {"name", "tags", "min_budget", "max_budget", 
                "duration_hours", "suitable_for", "description"}
    if not expected.issubset(experiences.columns):
        missing = expected - set(experiences.columns)
        raise KeyError(f"Missing required columns: {missing}")

    # Preprocess columns: lowercase tags & suitable_for lists
    df = experiences.assign(
        tags=experiences["tags"].map(lambda tags: [t.lower().strip() for t in tags]),
        suitable_for=experiences["suitable_for"].map(lambda grp: [g.lower().strip() for g in grp])
    )

    user_interests = set(i.lower().strip() for i in user_profile.get("interests", []))
    budget = user_profile.get("budget_aed", 0)
    group_type = user_profile.get("group_type", "").lower()
    duration_days = float(user_profile.get("duration_days", 0))
    available_hours = duration_days * 8

    def _compute_score(row):
        score = 0
        nodes = []

        # Interest overlap → 10 pts per matched tag (max 40)
        overlap = user_interests.intersection(set(row["tags"]))
        i_score = min(len(overlap) * 10, 40)
        score += i_score
        if overlap:
            nodes.append(f"Interest match: {', '.join(sorted(overlap))} +{i_score}")

        # Budget fit → 30 points if within range, else 0
        if row["min_budget"] <= budget <= row["max_budget"]:
            score += 30
            nodes.append(f"Budget {budget} within [{row['min_budget']}-{row['max_budget']}] +30")
        else:
            nodes.append(f"Budget {budget} outside range +0")

        # Duration check → 20 points if fits
        if row["duration_hours"] <= available_hours:
            score += 20
            nodes.append(f"Duration {row['duration_hours']}h fits in {available_hours}h +20")
        else:
            nodes.append(f"Duration {row['duration_hours']}h too long +0")

        # Group suitability → 10 points if group type matches
        if group_type in row["suitable_for"]:
            score += 10
            nodes.append(f"Group '{group_type}' is suitable +10")
        else:
            nodes.append(f"Group '{group_type}' not a match +0")

        return pd.Series({"score": score, "rationale": "; ".join(nodes)})

    # Apply scoring
    scored = df.apply(_compute_score, axis=1)
    df = df.join(scored)

    # Sort by score desc and return as list of dicts
    df_sorted = df.sort_values(by="score", ascending=False)

    return df_sorted[["name", "description", "score", "rationale"]].to_dict(orient="records")


In [32]:
user_profile = {
    "interests": ["culture", "shopping", "beach"],
    "budget_aed": 600,
    "duration_days": 2,
    "group_type": "family"
}

recommendations = match_experiences_to_profile(user_profile, dubai_tourist_spots)


### 7. Product Recommendation Layer

In [34]:
def product_recommendation_layer(
    scored_recs: List[Dict[str, Any]],
    top_n: int = 3
) -> Tuple[str, str]:
    """
    Select and format top-N recommended experiences for user output.

    Parameters:
      - scored_recs (List[dict]): List of experiences sorted descending by 'score',
        where each dict has keys: 'name', 'description', 'score', 'rationale'.
      - top_n (int): Number of top items to include in the final message.

    Returns:
      - message (str): A polished conversational recommendation prompt.
      - top_recs_json (str): JSON string of top recommendations (for validation).
    """

    if not scored_recs:
        return (
            "Sorry, I couldn't find any experiences that matched your preferences. Could you share more details?",
            json.dumps([])
        )

    top_n = min(top_n, len(scored_recs))

    lines = [
        f"Here are the top {top_n} experiences I recommend:"
    ]

    top_recs = []

    for idx, rec in enumerate(scored_recs[:top_n], start=1):
        lines.append(f"\n{idx}. **{rec['name']}**")
        lines.append(f"   • Description: {rec['description']}")
        lines.append(f"   • Score: {rec['score']} points")
        lines.append(f"   • Why I picked it: {rec['rationale']}")
        
        # Add to output for validation
        top_recs.append({
            "name": rec["name"],
            "description": rec["description"],
            "score": rec["score"],
            "rationale": rec["rationale"]
        })

    lines.append("\nWould you like to book one of these, or explore more options?")

    message = "\n".join(lines)
    return message, json.dumps(top_recs)

### 8. Validate recommendations atleast with basic score

In [36]:
def recommendation_validation(top3_recommendations_json):
    try:
        data = json.loads(top3_recommendations_json)
        return [r for r in data if isinstance(r, dict) and r.get("score", 0) > 10]
    except Exception as e:
        print("Validation error:", e)
        return []


### 9. Usage of function description calling APIs for profile matching

In [38]:
## Function Description for the Function Calling API
function_descriptions = [
    {
        "name": "match_experiences_to_profile",
        "description": "Get personalized Dubai experience recommendations based on user's travel interests, budget, available time, and group preferences. Scores and ranks the best-suited experiences.",
        "parameters": {
            "type": "object",
            "properties": {
                "interests": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    },
                    "description": "User’s top interests while visiting Dubai (e.g., 'culture', 'adventure', 'desert', 'shopping', 'luxury', 'nature'). These will be used to match experience tags."
                },
                "budget_aed": {
                    "type": "integer",
                    "description": "The total budget (in AED) that the user is willing to spend on experiences during their trip."
                },
                "duration_days": {
                    "type": "number",
                    "description": "Total number of days the user will spend in Dubai. Assumes 8 hours of experience time per day."
                },
                "group_type": {
                    "type": "string",
                    "enum": ["solo", "couple", "family", "friends", "group"],
                    "description": "The type of group traveling. Used to filter experiences that are suitable for the travel party."
                },
                "experience_type_preference": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    },
                    "description": "Optional. Categories or types of experiences the user prefers (e.g., 'museums', 'water activities', 'theme parks', 'historical tours'). Helps narrow down the matches further."
                },
                "strict_budget_match": {
                    "type": "boolean",
                    "description": "Optional. If true, only experiences fully within budget are considered. If false or omitted, partial matches are scored but not excluded."
                }
            },
            "required": ["interests", "budget_aed", "duration_days", "group_type"]
        }
    }
]


### 10. Dailogue Management System

In [40]:
def dialogue_mgmt_system():
    user_conversation = initialize_conversation()
    
    introduction = get_chat_model_completions(user_conversation)
    
    user_input = ""
    
    while(user_input != "exit"):
        user_input = input("")
        
        moderation = moderation_check(user_input)
        if 'Flagged' in moderation:
            print("Sorry, this message has been flagged. Please restart your conversation.")
            break
        
        user_conversation.append({"role": "user", "content": user_input})

        response_from_assistant = get_chat_model_completions(
                                        user_conversation,
                                        model="gpt-3.5-turbo",
                                        temperature=0.4,
                                        tools=[{"type": "function", "function": f} for f in function_descriptions],
                                        tool_choice="auto"
                                    )

        try:  
            if response_from_assistant["type"] == "tool_call":
                print("\nThank you for providing all the information. Kindly wait, while I fetch the details \n")
            
                # Step 2: Extract top3 Dubai Local attractions by calling the external function
                function_name  = response_from_assistant["tool_name"]
                function_args  = response_from_assistant["tool_args"]
                print(f"User requirement: {function_args }")
                recommendations = match_experiences_to_profile(function_args)
                print("scores: \n",recommendations[0])
                top3_recommendations, scores_dict = product_recommendation_layer(recommendations)
                print("Top3 attractions: \n",top3_recommendations)
                function_response = recommendation_validation(scores_dict)
                if len(function_response) == 0:
                    print("Sorry, we do not have any local dubai attractions that match your requirements. Connecting you to a human expert.")
                    break
                # Step 3: send the info on the function call and function response to GPT
                user_conversation.append({
                                            "role": "assistant",
                                            "content": None,
                                            "tool_calls": [
                                                {
                                                    "id": response_from_assistant["tool_call_id"],  
                                                    "function": {
                                                        "name": response_from_assistant["tool_name"],  
                                                        "arguments": json.dumps(response_from_assistant["tool_args"]) 
                                                    },
                                                    "type": "function"
                                                }
                                            ]
                                        })

                user_conversation.append(
                    {
                        
                        "role": "tool",
                        "tool_call_id": response_from_assistant["tool_call_id"],
                        "content": json.dumps(function_response) 

                    }
                )
                recommendation = get_chat_model_completions(user_conversation)
                user_conversation.append({"role": "assistant", "content": recommendation["content"]})
            else:
                user_conversation.append({"role": "assistant", "content": response_from_assistant["content"]})
                print("\n" +  response_from_assistant["content"] + "\n")
        except Exception as e:
            print("Failed to process tool call:", e)
                 

In [41]:
dialogue_mgmt_system()


 Assistant Response:

Based on your preferences, here's a tailored itinerary for your day in Dubai:

Morning:
- Start at Al Fahidi Historical District to explore Dubai Museum. Cost: 30 AED per person.
- Take an abra across the Creek to the Gold and Spice Souks. Cost: 1 AED per person.
- Enjoy Emirati cuisine at Al Seef or Arabian Tea House. Budget: around 100 AED for two.

Total morning cost: 132 AED for two people.

Afternoon:
- Visit the Jumeirah Mosque for a cultural experience. Cost: 25 AED per person.
- Relax at Kite Beach and enjoy the views. Free entry.
- Walk along the JBR beachfront and explore The Beach boardwalk. Free activity.

Total afternoon cost: 50 AED for two people.

Evening:
- Have dinner at a budget-friendly restaurant near Dubai Marina. Budget: around 150 AED for two.
- Explore the Dubai Marina area and enjoy the evening ambiance.

Total evening cost: 150 AED for two people.

Overall estimated cost for the day: 332 AED for two people. This plan covers cultural sit

 I want to explore historical sites and shop for souvenirs. I'm coming with my family for 3 days and want to spend about 800 AED



 Assistant wants to call: match_experiences_to_profile
With arguments: {'interests': ['culture', 'shopping'], 'budget_aed': 800, 'duration_days': 3, 'group_type': 'family', 'strict_budget_match': True}

Thank you for providing all the information. Kindly wait, while I fetch the details 

User requirement: {'interests': ['culture', 'shopping'], 'budget_aed': 800, 'duration_days': 3, 'group_type': 'family', 'strict_budget_match': True}
scores: 
 {'name': 'Desert Safari', 'description': 'Dune‑bashing, camel rides, traditional dinner under the stars.', 'score': 20, 'rationale': "Budget 800 outside range +0; Duration 6.0h fits in 24.0h +20; Group 'family' not a match +0"}
Top3 attractions: 
 Here are the top 3 experiences I recommend:

1. **Desert Safari**
   • Description: Dune‑bashing, camel rides, traditional dinner under the stars.
   • Score: 20 points
   • Why I picked it: Budget 800 outside range +0; Duration 6.0h fits in 24.0h +20; Group 'family' not a match +0

2. **Motiongate Dub

 exit



 Assistant Response:

If you have any more questions in the future or need assistance with exploring Dubai, feel free to return. Have a great day! Goodbye!

If you have any more questions in the future or need assistance with exploring Dubai, feel free to return. Have a great day! Goodbye!



In [42]:
def evaluate_model_response(tagged_dict, model_dict):
    """
    Compare the reference (tagged) user profile dict with the model's extracted profile dict.
    Returns evaluation metrics and mismatch details.
    """

    metrics = {"total_fields": 0, "correct": 0, "incorrect": []}

    for key in ["interests", "budget_aed", "duration_days", "group_type"]:
        metrics["total_fields"] += 1
        ref = tagged_dict.get(key)
        pred = model_dict.get(key)

        if ref == pred:
            metrics["correct"] += 1
        else:
            metrics["incorrect"].append({"field": key, "expected": ref, "predicted": pred})

    accuracy = metrics["correct"] / metrics["total_fields"]
    return {
        "accuracy": accuracy,
        "correct_fields": metrics["correct"],
        "total_fields": metrics["total_fields"],
        "mismatches": metrics["incorrect"]
    }


In [43]:
tagged = {
    "interests": ["culture", "food"], 
    "budget_aed": 600, 
    "duration_days": 2, 
    "group_type": "family"
}
model = {
    "interests": ["culture", "shopping"], 
    "budget_aed": 600, 
    "duration_days": 2, 
    "group_type": "family"
}

result = evaluate_model_response(tagged, model)
print(result)

{'accuracy': 0.75, 'correct_fields': 3, 'total_fields': 4, 'mismatches': [{'field': 'interests', 'expected': ['culture', 'food'], 'predicted': ['culture', 'shopping']}]}


### Future Scope of Work for Dubai Local Explorer AI Chatbot

1. **Context-Aware Memory & Session Persistence**

    ***Objective:*** Implement a memory module to retain user profiles, preferences, and past interactions across sessions.

    ***Implementation:***

    - Utilize technologies like Redis or vector databases (e.g., Pinecone, FAISS) for session storage.
    
    - Ensure secure handling of user data, complying with GDPR and local data protection regulations.
    
    - Provide users with options to view, update, or delete their stored information.

2. **Live Integration with Real-Time APIs**

    ***Objective:*** Enhance the chatbot's recommendations with up-to-date information.

    ***Implementation:***

    - Integrate with Dubai's tourism APIs to fetch real-time data on attractions, events, and experiences.
    
    - Incorporate weather APIs to adjust recommendations based on current conditions.
    
    - Connect with booking platforms to provide users with seamless reservation options.

3. ***Feedback Learning Loop***

    ***Objective:*** Enable the chatbot to learn from user feedback and improve over time.

    ***Implementation:***
    
    - Implement thumbs-up/thumbs-down feedback mechanisms after each recommendation.
    
    - Analyze feedback to identify areas for improvement in the chatbot's responses.
    
    - Use insights to fine-tune the chatbot's algorithms and enhance user satisfaction.

4. **Natural Language Understanding (NLU) Enhancements**

    ***Objective:*** Improve the chatbot's understanding of user inputs.
    
    ***Implementation:***
    
    - Fine-tune NLU models to better handle diverse user queries and intents.
    
    - Implement multi-intent detection to process complex user requests.
    
    - Enhance disambiguation capabilities to clarify user inputs when necessary.

5. **Multilingual Support**

    ***Objective:*** Cater to Dubai's diverse population by supporting multiple languages.
    
    ***Implementation:***
    
    - Integrate translation APIs to support languages such as Arabic, Hindi, Urdu, and Chinese.
    
    - Ensure seamless language switching within conversations.
    
    - Adapt cultural nuances and idiomatic expressions for each supported language.

6. **Smart Itinerary Generator**

    ***Objective:*** Provide users with personalized travel itineraries.
    
    ***Implementation:***
    
    - Develop algorithms to generate itineraries based on user preferences, budget, and duration of stay.
    
    - Incorporate real-time data to adjust itineraries dynamically.
    
    - Allow users to customize and save their itineraries for future reference.

7. **Voice + WhatsApp Integration**

    ***Objective:*** Expand the chatbot's reach through voice and messaging platforms.
    
    ***Implementation:***
    
    - Integrate with voice recognition APIs to support voice inputs.
    
    - Connect with WhatsApp Business API to facilitate conversations via messaging.
    
    - Ensure consistent user experience across different platforms.

8. **User Authentication & Dashboard**

    ***Objective:*** Provide users with personalized experiences and control over their data.
    
    ***Implementation:***
    
    - Implement user authentication using OAuth or JWT tokens.
    
    - Develop a dashboard for users to manage their profiles, preferences, and saved itineraries.
    
    - Ensure data privacy and security in line with best practices.

9. **AI Reasoning Evaluation Suite**

    ***Objective:*** Continuously assess and improve the chatbot's reasoning capabilities.
    
    ***Implementation:***
    
    - Develop evaluation metrics to assess the accuracy and relevance of the chatbot's responses.
    
    - Implement logging and monitoring tools to track performance over time.
    
    - Use insights to refine algorithms and enhance decision-making processes.

10. **Scalability and Deployment**

    ***Objective:*** Ensure the chatbot can handle increased user traffic and data.
    
    ***Implementation:***
    
    - Containerize the application using Docker for consistent deployment environments.
    
    - Deploy using Kubernetes to manage scaling and orchestration.
    
    - Implement load balancing and auto-scaling to handle varying traffic loads.

11. **UX & UI Enhancements**

    ***Objective:*** Improve the user interface and experience.
    
    ***Implementation:***
    
    - Redesign the chatbot interface to be more intuitive and user-friendly.
    
    - Implement features like typing indicators, quick replies, and rich media support.
    
    - Ensure responsiveness across different devices and screen sizes.

12. **Security, Moderation & Data Privacy**

    ***Objective:*** Protect user data and ensure compliance with regulations.
    
    ***Implementation:***
    
    - Implement data encryption for all user interactions.
    
    - Integrate moderation tools to filter inappropriate content.
    
    - Regularly audit and update security protocols to address emerging threats.

**Additional Considerations**

- **Ethical AI Practices:** Adhere to Dubai's AI Ethics Guidelines to ensure transparency, fairness, and accountability in AI-driven decisions.

- **User-Centric Design:** Continuously gather user feedback to inform iterative improvements and ensure the chatbot meets user needs and expectations.

- **Collaboration with Local Entities:** Partner with local tourism boards, businesses, and government agencies to provide users with accurate and comprehensive information.